<a href="https://colab.research.google.com/github/JannaBabicheva/hw08_modules/blob/main/Babicheva_hw08_modules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credits: this notebook belongs to [Practical DL](https://docs.google.com/forms/d/e/1FAIpQLScvrVtuwrHSlxWqHnLt1V-_7h2eON_mlRR6MUb3xEe5x9LuoA/viewform?usp=sf_link) course by Yandex School of Data Analysis.

In [1]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

Module - это абстрактный класс, который определяет фундаментальные методы, необходимые для обучения нейронной сети. Вам не нужно ничего здесь менять, просто прочитайте комментарии.

In [2]:
class Module(object):
    """
    В принципе, вы можете думать о модуле как о чем-то (черном ящике).
    который может обрабатывать "входные" данные и выдавать "выходные" данные.
    Это похоже на применение функции, которая называется "переадресация`:

        output = module.forward(input)

    Модуль должен уметь выполнять обратный переход, чтобы отличать функцию "вперед".
    Более того, он должен уметь отличать ее, если она является частью цепочки (правило цепочки).
    Последнее подразумевает, что существует отклонение от предыдущего шага правила цепочки.

         gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True

    def forward(self, input):
        """
        Принимает входной объект и вычисляет соответствующий выходной сигнал модуля.
        """
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        """
Выполняет шаг обратного распространения по модулю относительно заданного входного сигнала.

        Это включает в себя: - вычисление градиента с использованием "входных данных" (необходимо для дальнейшей обратной обработки),
- вычисление градиента с параметрами (для обновления параметров при оптимизации).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput


    def updateOutput(self, input):
        """
       Вычисляет выходные данные, используя текущий набор параметров класса и input.
        Эта функция возвращает результат, который сохраняется в поле "output".

        Убедитесь, что данные как сохраняются в поле "output", так и возвращаются.
        """

        # The easiest case:

        # self.output = input
        # return self.output

        pass

    def updateGradInput(self, input, gradOutput):
        """
       Вычисление градиента модуля относительно его собственных входных данных.
        Это значение возвращается в `gradInput`. Кроме того, переменная состояния `gradInput` обновляется соответствующим образом.

        Форма `gradInput` всегда совпадает с формой `input`.

        Убедитесь, что вы сохранили градиенты в поле "gradInput" и вернули его.
        """

        # The easiest case:

        # self.gradInput = gradOutput
        # return self.gradInput

        pass

    def accGradParameters(self, input, gradOutput):
        """
        Вычисление градиента модуля относительно его собственных параметров.
        Нет необходимости переопределять, если модуль не имеет параметров (например, ReLU).
        """
        pass

    def zeroGradParameters(self):
        """
        Обнуляет переменную gradParams, если в модуле есть параметры.
        """
        pass

    def getParameters(self):
        """
        Возвращает список с его параметрами.
        Если модуль не имеет параметров, возвращает пустой список.
        """
        return []

    def getGradParameters(self):
        """
        Возвращает список с градиентами относительно его параметров.
        Если модуль не имеет параметров, возвращает пустой список.
        """
        return []

    def train(self):
        """
        Устанавливает режим обучения для модуля.
        Поведение при обучении и тестировании отличается в случае отсева и пакетной нормы.
        """
        self.training = True

    def evaluate(self):
        """
        Устанавливает режим оценки для модуля.
        Поведение при обучении и тестировании отличается в случае отсева и пакетной нормы.
        """
        self.training = False

    def __repr__(self):
        """
       Красиво напечатано. Должно быть переопределено в каждом модуле, если вы хотите
иметь читаемое описание.
        """
        return "Module"

# Sequential container

Определите процедуры прямого и обратного прохождения.

 прямой и обратный проход для контейнера Sequential, который последовательно обрабатывает данные через слои нейронной сети.

In [3]:
class Sequential(Module):
    """
    Этот класс реализует контейнер, который последовательно обрабатывает входные данные.

    input обрабатывается каждым модулем (слоем) в self.modules последовательно.
    Результирующий массив называется output.
    """

    def __init__(self):
        """
        Инициализация Sequential контейнера.
        Наследует базовый класс Module и создает пустой список модулей.
        """
        super(Sequential, self).__init__()
        self.modules = []

    def add(self, module):
        """
        Добавляет модуль в контейнер.

        Args:
            module: модуль для добавления в последовательность
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Реализует прямой проход через последовательность модулей.

        Схема:
        y_0 = module[0].forward(input)
        y_1 = module[1].forward(y_0)
        ...
        output = module[n-1].forward(y_{n-2})

        Args:
            input: входные данные для первого модуля

        Returns:
            output: выход последнего модуля
        """
        # Начинаем с входных данных
        current_input = input

        # Сохраняем промежуточные выходы для обратного прохода
        self.intermediates = []

        # Проходим через каждый модуль последовательно
        for module in self.modules:
            # Сохраняем текущий вход для обратного прохода
            self.intermediates.append(current_input)

            # Пропускаем через текущий модуль
            current_input = module.forward(current_input)

        # Сохраняем и возвращаем финальный выход
        self.output = current_input
        return self.output

    def backward(self, input, gradOutput):
        """
        Реализует обратный проход через последовательность модулей.

        Схема:
        g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
        g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
        ...
        g_1 = module[1].backward(y_0, g_2)
        gradInput = module[0].backward(input, g_1)

        Args:
            input: оригинальный вход Sequential модуля
            gradOutput: градиент от следующего модуля

        Returns:
            gradInput: градиент по входу
        """
        # Начинаем с градиента выхода
        current_gradient = gradOutput

        # Проходим через модули в обратном порядке
        for i in range(len(self.modules) - 1, -1, -1):
            # Определяем правильный вход для текущего модуля:
            # Для первого модуля используем оригинальный вход,
            # для остальных - сохраненные промежуточные значения
            if i == 0:
                current_input = input
            else:
                current_input = self.intermediates[i]

            # Вычисляем градиент через текущий модуль
            current_gradient = self.modules[i].backward(current_input, current_gradient)

        # Сохраняем и возвращаем финальный градиент
        self.gradInput = current_gradient
        return self.gradInput

    def zeroGradParameters(self):
        """
        Обнуляет градиенты параметров всех модулей.
        """
        for module in self.modules:
            module.zeroGradParameters()

    def getParameters(self):
        """
        Собирает все параметры в список.

        Returns:
            list: список параметров всех модулей
        """
        return [x.getParameters() for x in self.modules]

    def getGradParameters(self):
        """
        Собирает все градиенты параметров в список.

        Returns:
            list: список градиентов параметров всех модулей
        """
        return [x.getGradParameters() for x in self.modules]

    def __repr__(self):
        """
        Строковое представление последовательности модулей.

        Returns:
            str: строка с описанием всех модулей
        """
        string = "".join([str(x) + '\n' for x in self.modules])
        return string

    def __getitem__(self, x):
        """
        Получение модуля по индексу.

        Args:
            x: индекс модуля

        Returns:
            Module: модуль с указанным индексом
        """
        return self.modules.__getitem__(x)

    def train(self):
        """
        Переводит все модули в режим обучения.
        """
        self.training = True
        for module in self.modules:
            module.train()

    def evaluate(self):
        """
        Переводит все модули в режим оценки.
        """
        self.training = False
        for module in self.modules:
            module.evaluate()


1. Прямой проход:
   - Последовательное применение слоев
   - Сохранение промежуточных результатов
   - Каждый выход становится входом для следующего слоя

2. Обратный проход:
   - Проход в обратном порядке
   - Использование сохраненных промежуточных значений
   - Правильная передача градиентов между слоями

3. Управление состоянием:
   - Режимы train/evaluate
   - Сбор параметров и градиентов
   - Обнуление градиентов

4. Важные моменты:
   - Сохранение промежуточных результатов для backward
   - Правильная передача входов для каждого слоя
   - Поддержка всех необходимых операций (train, eval, параметры)

# Layers

## 1. Слой линейного преобразования
Также известен как плотный слой, полносвязанный слой, FC-слой, внутренний продуктовый слой InnerProductLayer (в кофе), аффинная трансформация - affine transform

- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [4]:
class Linear(Module):
    """
    Модуль, который применяет линейное преобразование.
    Работает с 2D-входом формы (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()

        # Инициализация весов и смещений
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size=(n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size=n_out)

        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def updateOutput(self, input):
        """
        Прямой проход: y = Wx + b
        input: [batch_size, n_feats1]
        W: [n_feats2, n_feats1]
        b: [n_feats2]
        output: [batch_size, n_feats2]
        """
        self.output = np.dot(input, self.W.T) + self.b
        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Обратный проход по входу.
        gradOutput: [batch_size, n_feats2]
        W: [n_feats2, n_feats1]
        gradInput: [batch_size, n_feats1]
        """
        self.gradInput = np.dot(gradOutput, self.W)
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        """
        Обратный проход по параметрам.
        input: [batch_size, n_feats1]
        gradOutput: [batch_size, n_feats2]
        gradW: [n_feats2, n_feats1]
        gradb: [n_feats2]
        """
        # Градиент по весам
        self.gradW = np.dot(gradOutput.T, input)

        # Градиент по смещениям
        self.gradb = np.sum(gradOutput, axis=0)

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1], s[0])
        return q


Основные компоненты реализации:

1. `updateOutput(self, input)`:
- Реализует прямой проход y = Wx + b
- Используется np.dot для матричного умножения
- Добавляется вектор смещения b
- Размерности: input [batch_size, n_feats1] -> output [batch_size, n_feats2]

2. `updateGradInput(self, input, gradOutput)`:
- Вычисляет градиент по входу
- Использует транспонированную матрицу весов
- Размерности: gradOutput [batch_size, n_feats2] -> gradInput [batch_size, n_feats1]

3. `accGradParameters(self, input, gradOutput)`:
- Вычисляет градиенты по параметрам (W и b)
- Для весов: gradW = gradOutput^T * input
- Для смещений: gradb = sum(gradOutput, axis=0)
- Размерности:
  - gradW: [n_feats2, n_feats1]
  - gradb: [n_feats2]

4. Вспомогательные методы:
- `zeroGradParameters()`: обнуляет градиенты
- `getParameters()`: возвращает текущие параметры
- `getGradParameters()`: возвращает градиенты параметров

Линейный слой является основным строительным блоком нейронных сетей.

## 2. SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

In [5]:
class SoftMax(Module):
    def __init__(self):
        super(SoftMax, self).__init__()

    def updateOutput(self, input):
        """
        Прямой проход:

        softmax(x)_i = exp(x_i) / sum_j(exp(x_j))

        input: [batch_size, n_feats]
        output: [batch_size, n_feats]
        """
        # Нормализация для численной стабильности
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))

        # Вычисляем exp() от нормализованных значений
        exp_x = np.exp(self.output)

        # Делим на сумму по строке для получения вероятностей
        self.output = exp_x / np.sum(exp_x, axis=1, keepdims=True)

        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Обратный проход.

        Градиент softmax:
        d(softmax_i)/d(x_j) = softmax_i * (delta_ij - softmax_j)
        где delta_ij = 1 если i=j, иначе 0

        input: [batch_size, n_feats]
        gradOutput: [batch_size, n_feats]
        gradInput: [batch_size, n_feats]
        """
        # Получаем значения softmax с прямого прохода
        softmax_output = self.output

        # Для каждого примера в батче
        self.gradInput = np.zeros_like(gradOutput)
        for i in range(len(gradOutput)):
            # Вычисляем якобиан softmax
            # diag(softmax) - outer(softmax, softmax)
            jacobian = np.diag(softmax_output[i]) - \
                      np.outer(softmax_output[i], softmax_output[i])

            # Умножаем градиент на якобиан
            self.gradInput[i] = np.dot(gradOutput[i], jacobian)

        return self.gradInput

    def __repr__(self):
        return "SoftMax"

## 3. LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

Основная цель этого уровня - использовать его при вычислении потерь с логарифмическим правдоподобием.

LogSoftMax применяет логарифм функции софтмакс к входным данным.

In [6]:
class LogSoftMax(Module):
    def __init__(self):
        super(LogSoftMax, self).__init__()

    def updateOutput(self, input):
        """
        Прямой проход: logsoftmax(x)_i = x_i - log(sum_j(exp(x_j)))

        input: [batch_size, n_feats]
        output: [batch_size, n_feats]
        """
        # Нормализация для численной стабильности
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))

        # Вычисляем exp() от нормализованных значений
        exp_x = np.exp(self.output)

        # Вычисляем логарифм суммы экспонент
        log_sum_exp = np.log(np.sum(exp_x, axis=1, keepdims=True))

        # Финальный результат: x_i - log(sum(exp(x_j)))
        self.output = self.output - log_sum_exp

        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Обратный проход.

        Градиент logsoftmax:
        d(logsoftmax_i)/d(x_j) = delta_ij - softmax_j
        где delta_ij = 1 если i=j, иначе 0

        input: [batch_size, n_feats]
        gradOutput: [batch_size, n_feats]
        gradInput: [batch_size, n_feats]
        """
        # Сначала вычисляем softmax от входа
        # Используем ту же нормализацию
        normalized_input = np.subtract(input, input.max(axis=1, keepdims=True))
        exp_x = np.exp(normalized_input)
        softmax_output = exp_x / np.sum(exp_x, axis=1, keepdims=True)

        # Градиент: gradOutput_i - sum_j(gradOutput_j * softmax_j)
        self.gradInput = gradOutput - np.sum(gradOutput, axis=1, keepdims=True) * softmax_output

        return self.gradInput

    def __repr__(self):
        return "LogSoftMax"

Особенности реализации:
1. Используется нормализация для предотвращения численной нестабильности
2. Градиент имеет более простую форму, чем у обычного softmax
3. LogSoftMax часто используется вместе с NLL (Negative Log Likelihood) loss для задач классификации
4. Эта комбинация численно более стабильна, чем использование обычного softmax с cross-entropy loss

LogSoftMax часто используется в последнем слое нейронной сети для задач классификации, особенно когда используется функция потерь NLL (отрицательное логарифмическое правдоподобие).

## 4. Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance:
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance.

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

 Batch Normalization


Перевод задания:


Одна из самых значимых недавних идей, которая сильно повлияла на нейронные сети - это пакетная нормализация. Идея проста, но эффективна: признаки должны быть отбелены (среднее=0, станд=1) на протяжении всей нейронной сети. Это улучшает сходимость для глубоких моделей, позволяя обучать их пару дней, а не недели. Вам нужно реализовать первую часть слоя: нормализацию признаков. Вторая часть (слой `ChannelwiseScaling`) реализована ниже.


Вход: `batch_size x n_feats`
Выход: `batch_size x n_feats`


Слой должен работать следующим образом. Во время обучения (`self.training == True`) он преобразует вход как:

$ y = (x - μ) / √(σ + ε) $

где μ и σ - среднее и дисперсия значений признаков в батче,

а ε - просто маленькое число для численной стабильности.


Также во время обучения слой должен поддерживать экспоненциальное скользящее среднее для значений среднего и дисперсии:

`'self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)'`


`self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)`


Во время тестирования (`self.training == False`) слой нормализует вход, используя `moving_mean` и `moving_variance`.



In [7]:
class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None
        self.moving_variance = None
        self.training = True

    def updateOutput(self, input):
        """
        Прямой проход batch normalization.

        input: [batch_size, n_feats]
        output: [batch_size, n_feats]
        """
        # Инициализируем moving_mean и moving_variance если это первый проход
        if self.moving_mean is None:
            self.moving_mean = np.zeros(input.shape[1])
            self.moving_variance = np.ones(input.shape[1])

        # Сохраняем входные данные для обратного прохода
        self.input = input

        if self.training:
            # Вычисляем среднее и дисперсию по батчу
            self.batch_mean = np.mean(input, axis=0)
            self.batch_var = np.var(input, axis=0)

            # Обновляем moving averages
            self.moving_mean = self.moving_mean * self.alpha + \
                             self.batch_mean * (1 - self.alpha)
            self.moving_variance = self.moving_variance * self.alpha + \
                                 self.batch_var * (1 - self.alpha)

            # Нормализуем используя batch statistics
            self.x_centered = input - self.batch_mean
            self.std = np.sqrt(self.batch_var + self.EPS)
            self.output = self.x_centered / self.std

        else:
            # В режиме тестирования используем moving averages
            self.output = (input - self.moving_mean) / \
                         np.sqrt(self.moving_variance + self.EPS)

        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Обратный проход batch normalization.

        input: [batch_size, n_feats]
        gradOutput: [batch_size, n_feats]
        gradInput: [batch_size, n_feats]
        """
        if self.training:
            batch_size = input.shape[0]

            # Градиент по x_centered
            grad_x_centered = gradOutput / self.std

            # Градиент по variance
            grad_var = np.sum(gradOutput * self.x_centered * -0.5 * \
                            (self.batch_var + self.EPS)**(-1.5), axis=0)

            # Градиент по mean
            grad_mean = np.sum(gradOutput * -1.0 / self.std, axis=0) + \
                       grad_var * np.mean(-2.0 * self.x_centered, axis=0)

            # Градиент по input
            self.gradInput = gradOutput / self.std + \
                            grad_var * 2.0 * self.x_centered / batch_size + \
                            grad_mean / batch_size
        else:
            # В режиме тестирования просто масштабируем градиент
            self.gradInput = gradOutput / \
                            np.sqrt(self.moving_variance + self.EPS)

        return self.gradInput

    def __repr__(self):
        return "BatchNormalization"

In [8]:
class ChannelwiseScaling(Module):
    """
       Реализует линейное преобразование входных данных  y = \gamma * x + \beta
       where \gamma, \beta - обучаемые векторы длины  x.shape[-1]
    """
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)

        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def updateOutput(self, input):
        self.output = input * self.gamma + self.beta
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.gamma
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        self.gradBeta = np.sum(gradOutput, axis=0)
        self.gradGamma = np.sum(gradOutput*input, axis=0)

    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)

    def getParameters(self):
        return [self.gamma, self.beta]

    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        return "ChannelwiseScaling"

Основные компоненты реализации:

1. `updateOutput(self, input)`:
   - В режиме обучения:
     - Вычисляет среднее и дисперсию по батчу
     - Обновляет экспоненциальные скользящие средние
     - Нормализует данные используя статистики батча
   - В режиме тестирования:
     - Использует накопленные moving averages для нормализации

2. `updateGradInput(self, input, gradOutput)`:
   - В режиме обучения:
     - Вычисляет градиенты по всем компонентам (mean, variance, input)
     - Учитывает все зависимости при обратном распространении
   - В режиме тестирования:
     - Просто масштабирует градиент используя moving variance

Этот слой значительно улучшает обучение глубоких нейронных сетей, уменьшая internal covariate shift и позволяя использовать более высокие learning rates

Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

Практические заметки. Если BatchNormalization  размещена после слоя линейного преобразования (включая плотный слой, свертки, масштабирование по каналам), который реализует функцию типа `y = weight * x + bias`, то добавление смещения становится бесполезным и может быть опущено, поскольку его эффект будет отброшен при вычитании пакетного среднего. Если пакетная нормализация (за которой следует масштабирование по каналам) помещается перед слоем, который увеличивает масштаб (включая `ReLU, LeakyReLU`), за которым следует любой слой линейного преобразования, то параметр `gamma` в масштабировании по каналам может быть заблокирован, поскольку он может быть поглощен слоем линейного преобразования.

## 5. Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

Перевод задания:
Реализуйте `dropout`. Идея и реализация действительно просты: просто умножьте вход на маску $Бернулли(p)$. Здесь $p$ - вероятность того, что элемент будет обнулен.

Это доказало свою эффективность как метод регуляризации и предотвращения коадаптации нейронов.

Во время обучения (`self.training == True`) он должен сэмплировать маску на каждой итерации (для каждого батча), обнулять элементы и умножать элементы на $ 1/(1-p)$. Последнее необходимо для сохранения средних значений признаков близкими к средним значениям, которые будут в тестовом режиме. При тестировании этот модуль должен реализовывать тождественное преобразование, т.е. `self.output = input.`

*   Вход: `batch_size x n_feats`
*   Выход: `batch_size x n_feats`


In [9]:
class Dropout(Module):
    def __init__(self, p=0.5):
        """
        p: вероятность обнуления элемента
        """
        super(Dropout, self).__init__()

        self.p = p
        self.mask = None
        self.training = True

    def updateOutput(self, input):
        """
        Прямой проход dropout.

        В режиме обучения:
        - Создает маску Бернулли
        - Умножает вход на маску
        - Масштабирует на 1/(1-p)

        В режиме тестирования:
        - Возвращает вход без изменений

        input: [batch_size, n_feats]
        output: [batch_size, n_feats]
        """
        if self.training:
            # Генерируем маску Бернулли (1-p) - вероятность сохранить элемент
            self.mask = np.random.binomial(1, 1-self.p, size=input.shape)

            # Масштабируем на 1/(1-p) для сохранения математического ожидания
            self.output = input * self.mask / (1 - self.p)
        else:
            # В режиме тестирования просто копируем вход
            self.output = input

        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Обратный проход dropout.

        В режиме обучения:
        - Умножает градиент на ту же маску и масштабирует

        В режиме тестирования:
        - Возвращает градиент без изменений

        input: [batch_size, n_feats]
        gradOutput: [batch_size, n_feats]
        gradInput: [batch_size, n_feats]
        """
        if self.training:
            # Используем ту же маску и масштабирование
            self.gradInput = gradOutput * self.mask / (1 - self.p)
        else:
            # В режиме тестирования просто копируем градиент
            self.gradInput = gradOutput

        return self.gradInput

    def __repr__(self):
        return f"Dropout(p={self.p})"

Особенности реализации:
1. Маска генерируется заново для каждого батча во время обучения
2. Масштабирование на 1/(1-p) обеспечивает одинаковое математическое ожидание на обучении и тестировании
3. В режиме тестирования слой не производит никаких изменений входа
4. Та же маска используется как для прямого, так и для обратного прохода

Dropout - это мощный метод регуляризации, который:
- Предотвращает переобучение
- Уменьшает зависимость между нейронами
- Приводит к более надежным признакам
- Может рассматриваться как неявный ансамбль моделей

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**):

Функции активации

Вот полный пример нелинейности исправленного линейного блока (он же ReLU).:

In [10]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()

    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput

    def __repr__(self):
        return "ReLU"

## 6. Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

Leaky ReLU - это модификация обычного ReLU, которая решает проблему "умирающих нейронов", добавляя небольшой наклон для отрицательных значений.

Функция определяется как:

$ f(x) = x $, если $x > 0$

$f(x) = slope * x$, если $x ≤ 0$

In [11]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        """
        slope: наклон для отрицательных значений
        """
        super(LeakyReLU, self).__init__()
        self.slope = slope

    def updateOutput(self, input):
        """
        Прямой проход Leaky ReLU.

        f(x) = x если x > 0
        f(x) = slope * x если x ≤ 0

        input: [batch_size, n_feats]
        output: [batch_size, n_feats]
        """
        # Сохраняем маску для обратного прохода
        self.mask = (input > 0)

        # Применяем Leaky ReLU
        self.output = np.where(self.mask,
                             input,  # если x > 0
                             input * self.slope)  # если x ≤ 0

        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Обратный проход Leaky ReLU.

        f'(x) = 1 если x > 0
        f'(x) = slope если x ≤ 0

        input: [batch_size, n_feats]
        gradOutput: [batch_size, n_feats]
        gradInput: [batch_size, n_feats]
        """
        # Используем сохраненную маску для определения градиента
        self.gradInput = np.where(self.mask,
                                gradOutput,  # если x > 0
                                gradOutput * self.slope)  # если x ≤ 0

        return self.gradInput

    def __repr__(self):
        return f"LeakyReLU(slope={self.slope})"

Основные компоненты реализации:

1. `updateOutput(self, input)`:
   - Создаем маску для положительных значений: `input > 0`
   - Используем np.where для применения разных преобразований:
     - Для x > 0: оставляем как есть
     - Для x ≤ 0: умножаем на slope

2. `updateGradInput(self, input, gradOutput)`:
   - Используем ту же маску для определения градиента:
     - Для x > 0: градиент = 1
     - Для x ≤ 0: градиент = slope

Преимущества Leaky ReLU:
1. Решает проблему "умирающих нейронов" обычного ReLU
2. Позволяет градиенту течь через отрицательную область
3. Сохраняет простоту вычислений
4. Помогает в случаях, когда ReLU систематически деактивирует нейроны

Рекомендации по выбору slope:
- Типичные значения: 0.01 - 0.1
- Меньшие значения (≈0.01) ближе к обычному ReLU
- Большие значения (≈0.1) дают более сильный градиент для отрицательных значений
- Можно экспериментировать со значением slope в зависимости от задачи

## 7. ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

ELU - это функция активации, которая похожа на ReLU, но имеет экспоненциальную составляющую для отрицательных значений:

f(x) = x, если x > 0

f(x) = α * (exp(x) - 1), если x ≤ 0

In [12]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        """
        Инициализация ELU

        Параметры:
        alpha: коэффициент масштабирования для отрицательных значений
        """
        super(ELU, self).__init__()
        self.alpha = alpha

    def updateOutput(self, input):
        """
        Прямой проход ELU.

        f(x) = x если x > 0
        f(x) = alpha * (exp(x) - 1) если x ≤ 0

        Параметры:
        input: входной тензор [batch_size, n_feats]

        Возвращает:
        output: выходной тензор [batch_size, n_feats]
        """
        # Сохраняем маску положительных значений для обратного прохода
        self.mask = (input > 0)

        # Копируем вход для модификации
        self.output = input.copy()

        # Применяем экспоненциальную функцию только к отрицательным значениям
        neg_mask = ~self.mask
        self.output[neg_mask] = self.alpha * (np.exp(input[neg_mask]) - 1)

        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Обратный проход ELU.

        f'(x) = 1 если x > 0
        f'(x) = alpha * exp(x) если x ≤ 0

        Параметры:
        input: входной тензор [batch_size, n_feats]
        gradOutput: градиент от следующего слоя [batch_size, n_feats]

        Возвращает:
        gradInput: градиент по входу [batch_size, n_feats]
        """
        # Инициализируем градиент
        self.gradInput = gradOutput.copy()

        # Для отрицательных значений умножаем градиент на производную ELU
        neg_mask = ~self.mask
        self.gradInput[neg_mask] *= self.alpha * np.exp(input[neg_mask])

        return self.gradInput

    def __repr__(self):
        return f"ELU(alpha={self.alpha})"


Подробный разбор реализации:

1. Конструктор `__init__`:
   ```python
   def __init__(self, alpha = 1.0):
   ```
   - Принимает параметр `alpha`, который определяет масштаб отрицательной части
   - По умолчанию alpha = 1.0, что является стандартным выбором

2. Прямой проход `updateOutput`:
   ```python
   self.mask = (input > 0)  # Маска для положительных значений
   self.output = input.copy()  # Копируем вход
   neg_mask = ~self.mask  # Маска для отрицательных значений
   self.output[neg_mask] = self.alpha * (np.exp(input[neg_mask]) - 1)
   ```
   - Создаем маску для разделения положительных и отрицательных значений
   - Для x > 0: оставляем значения без изменений
   - Для x ≤ 0: применяем формулу α(exp(x) - 1)

3. Обратный проход `updateGradInput`:
   ```python
   self.gradInput = gradOutput.copy()
   neg_mask = ~self.mask
   self.gradInput[neg_mask] *= self.alpha * np.exp(input[neg_mask])
   ```
   - Для x > 0: градиент равен входному градиенту (производная = 1)
   - Для x ≤ 0: градиент умножается на α*exp(x) (производная ELU)

Преимущества ELU:
1. Как и LeakyReLU, решает проблему "умирающих нейронов"
2. Имеет гладкую функцию (непрерывная первая производная)
3. Может давать более быструю сходимость
4. Естественным образом приводит к нулевому среднему активаций

Рекомендации по использованию:
- Стандартное значение α = 1.0 работает хорошо в большинстве случаев
- ELU особенно эффективен в глубоких сетях
- Может потребовать больше вычислений из-за экспоненты
- Хорошо работает с batch normalization

## 8. SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

SoftPlus - это гладкая аппроксимация функции ReLU. Она определяется как:

f(x) = ln(1 + exp(x))

Её производная:

f'(x) = 1 / (1 + exp(-x)) = sigmoid(x)

In [13]:
class SoftPlus(Module):
    def __init__(self):
        """
        Инициализация SoftPlus
        Не требует параметров
        """
        super(SoftPlus, self).__init__()

    def updateOutput(self, input):
        """
        Прямой проход SoftPlus.

        f(x) = ln(1 + exp(x))

        Для численной стабильности используем:
        f(x) = x + ln(1 + exp(-x)) при x > threshold
        f(x) = ln(1 + exp(x)) при x ≤ threshold

        Параметры:
        input: входной тензор [batch_size, n_feats]

        Возвращает:
        output: выходной тензор [batch_size, n_feats]
        """
        # Сохраняем вход для обратного прохода
        self.input = input

        # Порог для численной стабильности
        threshold = 20

        # Маска для больших значений
        large_values = input > threshold

        # Инициализируем выход
        self.output = np.zeros_like(input)

        # Для больших значений используем стабильную формулу
        self.output[large_values] = input[large_values] + \
                                  np.log(1 + np.exp(-input[large_values]))

        # Для остальных значений используем обычную формулу
        small_values = ~large_values
        self.output[small_values] = np.log(1 + np.exp(input[small_values]))

        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Обратный проход SoftPlus.

        f'(x) = 1 / (1 + exp(-x)) = sigmoid(x)

        Параметры:
        input: входной тензор [batch_size, n_feats]
        gradOutput: градиент от следующего слоя [batch_size, n_feats]

        Возвращает:
        gradInput: градиент по входу [batch_size, n_feats]
        """
        # Вычисляем сигмоиду от входа (производная SoftPlus)
        sigmoid = 1 / (1 + np.exp(-input))

        # Умножаем входной градиент на производную
        self.gradInput = gradOutput * sigmoid

        return self.gradInput

    def __repr__(self):
        return "SoftPlus"

Подробный разбор :

1. Конструктор `__init__`:
   ```python
   def __init__(self):
   ```
   - Не требует параметров, так как SoftPlus - это фиксированная функция

2. Прямой проход `updateOutput`:
   ```python
   threshold = 20  # Порог для численной стабильности
   large_values = input > threshold
   
   # Для больших x используем: x + ln(1 + exp(-x))
   self.output[large_values] = input[large_values] + \
                              np.log(1 + np.exp(-input[large_values]))
   
   # Для малых x используем: ln(1 + exp(x))
   small_values = ~large_values
   self.output[small_values] = np.log(1 + np.exp(input[small_values]))
   ```
   - Используем разные формулы для больших и малых значений для численной стабильности
   - Для больших x используем преобразованную формулу, чтобы избежать переполнения

3. Обратный проход `updateGradInput`:
   ```python
   # Вычисляем сигмоиду (производная SoftPlus)
   sigmoid = 1 / (1 + np.exp(-input))
   
   # Умножаем градиент на производную
   self.gradInput = gradOutput * sigmoid
   ```
   - Производная SoftPlus - это сигмоида
   - Умножаем входной градиент на производную согласно правилу цепи

Особенности SoftPlus:
1. Является гладкой версией ReLU
2. Всегда имеет положительную производную
3. Выход всегда положителен (как у ReLU)
4. При больших положительных x близка к x (как ReLU)
5. При больших отрицательных x близка к 0 (как ReLU)

Сравнение с ReLU:
- Преимущества:
  - Гладкая функция (все производные существуют)
  - Нет резкого перехода в нуле
  - Может быть полезна, когда нужна гладкость
- Недостатки:
  - Требует больше вычислений
  - Градиент всегда меньше 1
  - Отсутствие разреженности активаций (или отсутчвие того, что значительная часть выходов нейронов равна точно нулю)

SoftPlus часто используется:
- В вероятностных моделях
- Когда требуется гладкость
- В задачах, где важна непрерывность производных

# Criterions

Criterions are used to score the models answers.

Критерии используются для оценки ответов моделей.

In [14]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None

    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)

    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Criterion"

Дополнительные пояснения:
1. Criterions (Критерии) используются для оценки качества ответов модели, то есть для вычисления функции потерь (loss function).

2. Базовый класс Criterion определяет интерфейс для всех критериев с двумя основными операциями:
   - forward: вычисление значения функции потерь
   - backward: вычисление градиентов для обратного распространения

3. Структура класса предполагает, что реальная реализация будет происходить в методах:
   - updateOutput: реализация прямого прохода
   - updateGradInput: реализация обратного прохода

4. Методы forward и backward являются оберткам и не должны переопределяться в дочерних классах для поддержания согласованности интерфейса.

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

MSE (Mean Squared Error - среднеквадратичная ошибка).

MSE - это базовый критерий для задач регрессии, который вычисляет среднее квадратов разностей между предсказанными и фактическими значениями.

In [15]:
class MSECriterion(Criterion):
    def __init__(self):
        """
        Инициализация критерия MSE
        Наследует базовый класс Criterion
        """
        super(MSECriterion, self).__init__()

    def updateOutput(self, input, target):
        """
        Прямой проход MSE:
        MSE = (1/n) * Σ(y_pred - y_true)²

        Параметры:
        input (y_pred): предсказания модели [batch_size x n_feats]
        target (y_true): целевые значения [batch_size x n_feats]

        Возвращает:
        output: скалярное значение ошибки (усредненное по батчу)
        """
        # Вычисляем квадрат разности и суммируем
        self.output = np.sum(np.power(input - target, 2)) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        """
        Обратный проход MSE.
        Производная MSE: d(MSE)/d(input) = 2(y_pred - y_true)/n

        Параметры:
        input (y_pred): предсказания модели [batch_size x n_feats]
        target (y_true): целевые значения [batch_size x n_feats]

        Возвращает:
        gradInput: градиент по входу [batch_size x n_feats]
        """
        # Вычисляем градиент
        self.gradInput = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

Разберем детально:

1. Формула MSE:
```
MSE = (1/n) * Σ(y_pred - y_true)²
```
где:
- n - размер батча (input.shape[0])
- y_pred - предсказания модели (input)
- y_true - истинные значения (target)

2. Прямой проход (`updateOutput`):
```python
self.output = np.sum(np.power(input - target, 2)) / input.shape[0]
```
- `input - target`: вычисляем разность между предсказаниями и целевыми значениями
- `np.power(..., 2)`: возводим разности в квадрат
- `np.sum(...)`: суммируем все квадраты разностей
- `/input.shape[0]`: делим на размер батча для получения среднего

3. Обратный проход (`updateGradInput`):
```python
self.gradInput = (input - target) * 2 / input.shape[0]
```
- Производная MSE получается из цепного правила:
  - d(MSE)/d(input) = 2(y_pred - y_true)/n
- Множитель 2 появляется из производной квадрата
- Деление на input.shape[0] из-за усреднения

Особенности MSE:
1. Квадратичная функция потерь
   - Сильно штрафует большие отклонения
   - Меньше штрафует малые отклонения

2. Преимущества:
   - Простая и понятная метрика
   - Всегда неотрицательная
   - Дифференцируема везде
   - Выпуклая функция

3. Недостатки:
   - Чувствительна к выбросам
   - Может замедлять обучение при больших ошибках

4. Применение:
   - Задачи регрессии
   - Предсказание непрерывных величин
   - Когда важно сильно штрафовать большие отклонения

## 9. Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula,
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



Negative Log-Likelihood (NLL) для классификации- это функция потерь, которая измеряет производительность модели, где выход представляет собой вероятности классов. Для one-hot encoded - функция упрощается до -log(p_c), где p_c - предсказанная вероятность правильного класса.

In [16]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15  # маленькая константа для численной стабильности

    def __init__(self):
        """
        Инициализация критерия NLL
        """
        super(ClassNLLCriterionUnstable, self).__init__()

    def updateOutput(self, input, target):
        """
        Прямой проход NLL:
        L = -(1/N) * Σ(t_i * log(p_i))
        где t_i - one-hot target, p_i - предсказанные вероятности

        Параметры:
        input: предсказанные вероятности [batch_size x n_feats]
        target: one-hot encoded цели [batch_size x n_feats]

        Возвращает:
        output: скалярное значение ошибки (усредненное по батчу)
        """
        # Ограничиваем вероятности для избежания log(0)
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)

        # Вычисляем -log(p) для всех предсказаний
        log_probs = -np.log(input_clamp)

        # Так как target - one-hot, умножение на target выберет
        # только log_prob для правильного класса
        self.output = np.sum(log_probs * target) / input.shape[0]

        return self.output

    def updateGradInput(self, input, target):
        """
        Обратный проход NLL.
        Градиент: -(1/N) * (t_i/p_i)

        Параметры:
        input: предсказанные вероятности [batch_size x n_feats]
        target: one-hot encoded цели [batch_size x n_feats]

        Возвращает:
        gradInput: градиент по входу [batch_size x n_feats]
        """
        # Ограничиваем вероятности для численной стабильности
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)

        # Градиент NLL: -target/input для каждого элемента
        self.gradInput = -target / input_clamp / input.shape[0]

        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterionUnstable"


1. Прямой проход (`updateOutput`):
```python
# Ограничиваем вероятности
input_clamp = np.clip(input, self.EPS, 1 - self.EPS)

 **# Вычисляем -log(p)**
log_probs = -np.log(input_clamp)

 # Умножаем на target и усредняем по батчу
self.output = np.sum(log_probs * target) / input.shape[0]
```
- Используем `clip` для избежания log(0)
- Вычисляем отрицательный логарифм вероятностей
- Умножение на one-hot target выбирает только нужные вероятности
- Усредняем по батчу

2. Обратный проход (`updateGradInput`):
```python
input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
self.gradInput = -target / input_clamp / input.shape[0]
```
- Производная -log(x) = -1/x
- Умножаем на -target для выбора нужных классов
- Делим на размер батча

Особенности реализации:
1. Численная стабильность:
   - Используется `EPS = 1e-15` для предотвращения log(0)
   - Вероятности ограничены в диапазоне [EPS, 1-EPS]

2. One-hot encoding:
   - Target содержит 1 только для правильного класса
   - Упрощает вычисления, так как не нужно искать правильный класс

3. Усреднение по батчу:
   - Все значения делятся на размер батча
   - Это делает функцию потерь независимой от размера батча

Недостатки этой реализации:
1. Численная нестабильность при очень малых или больших вероятностях
2. Может давать неточные результаты при работе с softmax
3. Лучше использовать более стабильную версию, которая работает с логитами

Типичное использование:
1. После слоя softmax в задачах классификации
2. Когда выход модели - вероятности классов
3. В комбинации с регуляризацией для предотвращения переобучения

## 10. Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

Negative Log-Likelihood критерий.

Основное отличие от предыдущей версии в том, что вход уже представляет собой логарифмы вероятностей (после log-softmax), что делает вычисления численно более стабильными.

In [17]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        """
        Инициализация стабильного NLL критерия
        """
        super(ClassNLLCriterion, self).__init__()

    def updateOutput(self, input, target):
        """
        Прямой проход стабильного NLL.

        L = -(1/N) * Σ(t_i * log_p_i)
        где:
        - t_i - one-hot target
        - log_p_i - логарифмы вероятностей (вход уже в логарифмической форме)

        Параметры:
        input: логарифмы вероятностей [batch_size x n_feats]
        target: one-hot encoded цели [batch_size x n_feats]

        Возвращает:
        output: скалярное значение ошибки (усредненное по батчу)
        """
        # Так как target - one-hot, умножение на target выберет
        # только log_prob для правильного класса
        # Знак минус уже учтен в входных логарифмах вероятностей
        self.output = -np.sum(input * target) / input.shape[0]

        return self.output

    def updateGradInput(self, input, target):
        """
        Обратный проход стабильного NLL.

        Градиент: -(1/N) * t_i
        (так как вход уже в логарифмической форме)

        Параметры:
        input: логарифмы вероятностей [batch_size x n_feats]
        target: one-hot encoded цели [batch_size x n_feats]

        Возвращает:
        gradInput: градиент по входу [batch_size x n_feats]
        """
        # Градиент - это просто масштабированный target с обратным знаком
        self.gradInput = -target / input.shape[0]

        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterion"

Подробно:

1. Прямой проход (`updateOutput`):
```python
self.output = -np.sum(input * target) / input.shape[0]
```
- Вход (`input`) уже содержит логарифмы вероятностей
- `target` в one-hot кодировке выбирает нужные логарифмы
- Делим на размер батча для усреднения
- Знак минус нужен для получения loss (минимизируем отрицательный log-likelihood)

2. Обратный проход (`updateGradInput`):
```python
self.gradInput = -target / input.shape[0]
```
- Градиент простой, так как вход уже в логарифмической форме
- Просто масштабированный target с обратным знаком
- Делим на размер батча для согласованности с прямым проходом

Преимущества этой реализации над нестабильной версией:

1. Численная стабильность:
   - Нет вычисления логарифмов (они уже в входных данных)
   - Нет деления на маленькие вероятности
   - Меньше вычислительных операций

2. Простота:
   - Код короче и понятнее
   - Меньше возможностей для ошибок
   - Проще отлаживать

3. Эффективность:
   - Меньше операций
   - Лучше работает с градиентным спуском

Типичное использование:

1. После слоя LogSoftmax:
```python
model = Sequential()
model.add(LogSoftmax())
criterion = ClassNLLCriterion()
```

2. В задачах классификации:
```python
# Предположим, у нас есть логиты от модели
logits = model(x)
# Применяем LogSoftmax
log_probs = log_softmax(logits)
# Вычисляем loss
loss = criterion(log_probs, targets)
```

3. С градиентным спуском:
```python
# Прямой проход
loss = criterion(log_probs, targets)
# Обратный проход
grad = criterion.backward(log_probs, targets)
# Обновление весов
optimizer.step(grad)
```

Важно помнить:
1. Вход должен быть именно логарифмами вероятностей
2. Обычно используется вместе с LogSoftmax слоем
3. Target должен быть в one-hot кодировке
4. Этот критерий численно стабилен и предпочтителен над нестабильной версией

# Optimizers

### SGD optimizer with momentum
- `variables` - list of lists of variables (one list per layer)
- `gradients` - list of lists of current gradients (same structure as for `variables`, one array for each var)
- `config` - dict with optimization parameters (`learning_rate` and `momentum`)
- `state` - dict with optimizator state (used to save accumulated gradients)

In [18]:
def sgd_momentum(variables, gradients, config, state):
    # 'variables' and 'gradients' have complex structure, accumulated_grads will be stored in a simpler one
    state.setdefault('accumulated_grads', {})

    var_index = 0
    for current_layer_vars, current_layer_grads in zip(variables, gradients):
        for current_var, current_grad in zip(current_layer_vars, current_layer_grads):

            old_grad = state['accumulated_grads'].setdefault(var_index, np.zeros_like(current_grad))

            np.add(config['momentum'] * old_grad, config['learning_rate'] * current_grad, out=old_grad)

            current_var -= old_grad
            var_index += 1

## 11. [Adam](https://arxiv.org/pdf/1412.6980.pdf) optimizer
- `variables` - list of lists of variables (one list per layer)
- `gradients` - list of lists of current gradients (same structure as for `variables`, one array for each var)
- `config` - dict with optimization parameters (`learning_rate`, `beta1`, `beta2`, `epsilon`)
- `state` - dict with optimizator state (used to save 1st and 2nd moment for vars)

Formulas for optimizer:

Current step learning rate: $$\text{lr}_t = \text{learning_rate} * \frac{\sqrt{1-\beta_2^t}} {1-\beta_1^t}$$
First moment of var: $$\mu_t = \beta_1 * \mu_{t-1} + (1 - \beta_1)*g$$
Second moment of var: $$v_t = \beta_2 * v_{t-1} + (1 - \beta_2)*g*g$$
New values of var: $$\text{variable} = \text{variable} - \text{lr}_t * \frac{m_t}{\sqrt{v_t} + \epsilon}$$

In [19]:
def adam_optimizer(variables, gradients, config, state):
    state.setdefault('m', {})  # first moment vars
    state.setdefault('v', {})  # second moment vars
    state.setdefault('t', 0)   # timestamp
    state['t'] += 1

    # Проверяем наличие всех необходимых параметров
    for k in ['learning_rate', 'beta1', 'beta2', 'epsilon']:
        assert k in config, config.keys()

    var_index = 0
    # Вычисляем корректированный learning rate
    lr_t = config['learning_rate'] * np.sqrt(1 - config['beta2']**state['t']) / \
           (1 - config['beta1']**state['t'])

    for current_layer_vars, current_layer_grads in zip(variables, gradients):
        for current_var, current_grad in zip(current_layer_vars, current_layer_grads):
            # Получаем моменты или инициализируем нулями
            var_first_moment = state['m'].setdefault(var_index, np.zeros_like(current_grad))
            var_second_moment = state['v'].setdefault(var_index, np.zeros_like(current_grad))

            # Обновляем первый момент (m)
            # m = beta1 * m + (1 - beta1) * g
            np.add(config['beta1'] * var_first_moment,
                  (1 - config['beta1']) * current_grad,
                  out=var_first_moment)

            # Обновляем второй момент (v)
            # v = beta2 * v + (1 - beta2) * g * g
            np.add(config['beta2'] * var_second_moment,
                  (1 - config['beta2']) * current_grad * current_grad,
                  out=var_second_moment)

            # Обновляем переменную
            # var -= lr_t * m / (sqrt(v) + epsilon)
            current_var -= lr_t * var_first_moment / \
                          (np.sqrt(var_second_moment) + config['epsilon'])

            assert var_first_moment is state['m'].get(var_index)
            assert var_second_moment is state['v'].get(var_index)
            var_index += 1

Сравнение оптимизаторов:

1. SGD с моментом:
- Плюсы:
  - Простая реализация
  - Меньше гиперпараметров
  - Хорошо работает для многих задач
- Минусы:
  - Единый learning rate для всех параметров
  - Может застревать в седловых точках
  - Требует ручной настройки learning rate

2. Adam:
- Плюсы:
  - Адаптивный learning rate для каждого параметра
  - Хорошо работает с разреженными градиентами
  - Меньше проблем с выбором learning rate
  - Эффективная коррекция смещения
- Минусы:
  - Более сложная реализация
  - Больше гиперпараметров
  - Может иметь проблемы с обобщением

Ключевые различия в реализации:

1. Накопление информации:
   - SGD с моментом: один накопленный градиент
   - Adam: два момента (среднее и дисперсия градиентов)

2. Обновление параметров:
   - SGD с моментом:
     ```python
     v = momentum * v + learning_rate * gradient
     var -= v
     ```
   - Adam:
     ```python
     m = beta1 * m + (1 - beta1) * gradient
     v = beta2 * v + (1 - beta2) * gradient^2
     var -= lr_t * m / (sqrt(v) + epsilon)
     ```

3. Learning rate:
   - SGD с моментом: фиксированный
   - Adam: адаптивный для каждого параметра и корректируется по времени

Типичные значения параметров:

1. SGD с моментом:
   ```python
   config = {
       'learning_rate': 0.01,
       'momentum': 0.9
   }
   ```

2. Adam:
   ```python
   config = {
       'learning_rate': 0.001,
       'beta1': 0.9,
       'beta2': 0.999,
       'epsilon': 1e-8
   }
   ```

learning rate

 В SGD с моментом используется один и тот же коэффициент learning rate для обновления всех параметров сети, независимо от их:
- важности
- масштаба
- частоты изменения
- градиентов

В то время как Adam адаптивно подбирает индивидуальный темп обучения для каждого параметра на основе:
- истории градиентов (первый момент)
- их изменчивости (второй момент)

Например, если у одного параметра градиенты стабильно большие, а у другого маленькие, Adam автоматически уменьшит шаг для первого и увеличит для второго. SGD же будет использовать одинаковый шаг для обоих.

# Layers for advanced track homework
You **don't need** to implement it if you are working on `homework_main-basic.ipynb`

## 12. Conv2d [Advanced]
- input:   **`batch_size x in_channels x h x w`**
- output: **`batch_size x out_channels x h x w`**

You should implement something like pytorch `Conv2d` layer with `stride=1` and zero-padding outside of image using `scipy.signal.correlate` function.

Practical notes:
- While the layer name is "convolution", the most of neural network frameworks (including tensorflow and pytorch) implement operation that is called [correlation](https://en.wikipedia.org/wiki/Cross-correlation#Cross-correlation_of_deterministic_signals) in signal processing theory. So **don't use** `scipy.signal.convolve` since it implements [convolution](https://en.wikipedia.org/wiki/Convolution#Discrete_convolution) in terms of signal processing.
- It may be convenient to use `skimage.util.pad` for zero-padding.
- It's rather ok to implement convolution over 4d array using 2 nested loops: one over batch size dimension and another one over output filters dimension
- Having troubles with understanding how to implement the layer?
 - Check the last year video of lecture 3 (starting from ~1:14:20)
 - May the google be with you

In [20]:
import scipy as sp
import scipy.signal
import skimage.util
import numpy as np

class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        """
        Инициализация слоя Conv2d.

        Параметры:
        in_channels: количество входных каналов
        out_channels: количество выходных каналов
        kernel_size: размер ядра свертки (нечетное число)
        """
        super(Conv2d, self).__init__()
        assert kernel_size % 2 == 1, kernel_size

        # Инициализация весов и смещений
        stdv = 1./np.sqrt(in_channels)
        self.W = np.random.uniform(-stdv, stdv,
                                 size=(out_channels, in_channels, kernel_size, kernel_size))
        self.b = np.random.uniform(-stdv, stdv, size=(out_channels,))

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        # Градиенты
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def updateOutput(self, input):
        """
        Прямой проход свертки.

        input: [batch_size, in_channels, h, w]
        output: [batch_size, out_channels, h, w]
        """
        pad_size = self.kernel_size // 2
        batch_size = input.shape[0]
        h, w = input.shape[2], input.shape[3]

        # Добавляем zero-padding к входу
        padded_input = np.pad(input,
                            ((0,0), (0,0), (pad_size,pad_size), (pad_size,pad_size)),
                            mode='constant')

        # Инициализируем выходной тензор
        self.output = np.zeros((batch_size, self.out_channels, h, w))

        # Выполняем свертку для каждого образца в батче и каждого выходного канала
        for i in range(batch_size):
            for j in range(self.out_channels):
                # Суммируем результаты свертки по всем входным каналам
                conv_sum = np.zeros((h, w))
                for k in range(self.in_channels):
                    conv_sum += sp.signal.correlate2d(padded_input[i,k],
                                                    self.W[j,k],
                                                    mode='valid')
                self.output[i,j] = conv_sum + self.b[j]

        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Обратный проход - вычисление градиента по входу.

        input: [batch_size, in_channels, h, w]
        gradOutput: [batch_size, out_channels, h, w]
        gradInput: [batch_size, in_channels, h, w]
        """
        pad_size = self.kernel_size // 2
        batch_size = input.shape[0]

        # Инициализируем градиент по входу
        self.gradInput = np.zeros_like(input)

        # Добавляем padding к градиенту выхода
        padded_gradOutput = np.pad(gradOutput,
                                 ((0,0), (0,0), (pad_size,pad_size), (pad_size,pad_size)),
                                 mode='constant')

        # Вычисляем градиент для каждого образца и входного канала
        for i in range(batch_size):
            for k in range(self.in_channels):
                grad_sum = np.zeros_like(self.gradInput[i,k])
                for j in range(self.out_channels):
                    # Поворачиваем ядро на 180 градусов для свертки
                    kernel_rotated = np.rot90(self.W[j,k], 2)
                    grad_sum += sp.signal.correlate2d(padded_gradOutput[i,j],
                                                    kernel_rotated,
                                                    mode='valid')
                self.gradInput[i,k] = grad_sum

        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        """
        Вычисление градиентов по параметрам (веса и смещения).

        input: [batch_size, in_channels, h, w]
        gradOutput: [batch_size, out_channels, h, w]
        """
        pad_size = self.kernel_size // 2
        batch_size = input.shape[0]

        # Добавляем padding к входу
        padded_input = np.pad(input,
                             ((0,0), (0,0), (pad_size,pad_size), (pad_size,pad_size)),
                             mode='constant')

        # Вычисляем градиенты весов
        for j in range(self.out_channels):
            for k in range(self.in_channels):
                grad_sum = np.zeros_like(self.W[j,k])
                for i in range(batch_size):
                    grad_sum += sp.signal.correlate2d(padded_input[i,k],
                                                    gradOutput[i,j],
                                                    mode='valid')
                self.gradW[j,k] = grad_sum

        # Вычисляем градиенты смещений
        self.gradb = np.sum(gradOutput, axis=(0,2,3))

    def zeroGradParameters(self):
        """Обнуляем градиенты"""
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        """Возвращаем параметры слоя"""
        return [self.W, self.b]

    def getGradParameters(self):
        """Возвращаем градиенты параметров"""
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        return f'Conv2d {s[1]} -> {s[0]}'

Ключевое:

1. Прямой проход (`updateOutput`):
```python
  # Добавляем padding
padded_input = np.pad(input,
                     ((0,0), (0,0), (pad_size,pad_size), (pad_size,pad_size)),
                     mode='constant')

 # Для каждого образца и выходного канала
for i in range(batch_size):
    for j in range(self.out_channels):
        conv_sum = np.zeros((h, w))
        for k in range(self.in_channels):
            conv_sum += sp.signal.correlate2d(padded_input[i,k],
                                            self.W[j,k],
                                            mode='valid')
        self.output[i,j] = conv_sum + self.b[j]
```

2. Обратный проход по входу (`updateGradInput`):
```python
# Для каждого образца и входного канала
for i in range(batch_size):
    for k in range(self.in_channels):
        grad_sum = np.zeros_like(self.gradInput[i,k])
        for j in range(self.out_channels):
            kernel_rotated = np.rot90(self.W[j,k], 2)
            grad_sum += sp.signal.correlate2d(padded_gradOutput[i,j],
                                            kernel_rotated,
                                            mode='valid')
        self.gradInput[i,k] = grad_sum
```

3. Вычисление градиентов параметров (`accGradParameters`):
```python
# Градиенты весов
for j in range(self.out_channels):
    for k in range(self.in_channels):
        grad_sum = np.zeros_like(self.W[j,k])
        for i in range(batch_size):
            grad_sum += sp.signal.correlate2d(padded_input[i,k],
                                            gradOutput[i,j],
                                            mode='valid')
        self.gradW[j,k] = grad_sum

 # Градиенты смещений
self.gradb = np.sum(gradOutput, axis=(0,2,3))
```

Важные особенности:
1. Используется корреляция вместо свертки
2. Добавляется zero-padding для сохранения размерности
3. Отдельно обрабатывается каждый канал
4. Учитываются смещения (bias) для каждого выходного канала

## 13. MaxPool2d [Advanced]
- input:   **`batch_size x n_input_channels x h x w`**
- output: **`batch_size x n_output_channels x h // kern_size x w // kern_size`**

You are to implement simplified version of pytorch `MaxPool2d` layer with stride = kernel_size. Please note, that it's not a common case that stride = kernel_size: in AlexNet and ResNet kernel_size for max-pooling was set to 3, while stride was set to 2. We introduce this restriction to make implementation simplier.

Practical notes:
- During forward pass what you need to do is just to reshape the input tensor to `[n, c, h / kern_size, kern_size, w / kern_size, kern_size]`, swap two axes and take maximums over the last two dimensions. Reshape + axes swap is sometimes called space-to-batch transform.
- During backward pass you need to place the gradients in positions of maximal values taken during the forward pass
- In real frameworks the indices of maximums are stored in memory during the forward pass. It is cheaper than to keep the layer input in memory and recompute the maximums.

In [21]:
class MaxPool2d(Module):
   def __init__(self, kernel_size):
       """
       Инициализация MaxPool2d
       kernel_size: размер окна для пулинга
       """
       super(MaxPool2d, self).__init__()
       self.kernel_size = kernel_size
       self.gradInput = None

   def updateOutput(self, input):
       """
       Прямой проход MaxPool2d

       input: [batch_size, channels, height, width]
       output: [batch_size, channels, height/kernel_size, width/kernel_size]
       """
       # Получаем размеры входа
       batch_size, channels, input_h, input_w = input.shape

       # Проверяем, что размеры делятся на размер ядра
       assert input_h % self.kernel_size == 0
       assert input_w % self.kernel_size == 0

       # Вычисляем выходные размеры
       output_h = input_h // self.kernel_size
       output_w = input_w // self.kernel_size

       # Преобразуем вход для удобного пулинга
       # [batch, channel, h, w] ->
       # [batch, channel, h/k, k, w/k, k]
       x_reshaped = input.reshape(batch_size, channels,
                                output_h, self.kernel_size,
                                output_w, self.kernel_size)

       # Меняем оси для группировки пространственных измерений
       # [batch, channel, h/k, k, w/k, k] ->
       # [batch, channel, h/k, w/k, k, k]
       x_reordered = x_reshaped.transpose(0, 1, 2, 4, 3, 5)

       # Получаем вид для операции максимума
       x_pooling = x_reordered.reshape(*x_reordered.shape[:-2], -1)

       # Находим индексы максимумов
       self.max_indices = x_pooling.argmax(axis=-1)

       # Находим максимальные значения
       self.output = x_pooling.max(axis=-1)

       return self.output

   def updateGradInput(self, input, gradOutput):
       """
       Обратный проход MaxPool2d

       input: [batch_size, channels, height, width]
       gradOutput: [batch_size, channels, height/kernel_size, width/kernel_size]
       gradInput: [batch_size, channels, height, width]
       """
       batch_size, channels, input_h, input_w = input.shape
       output_h = input_h // self.kernel_size
       output_w = input_w // self.kernel_size
       k = self.kernel_size

       # Создаем градиент входа нужной формы
       self.gradInput = np.zeros_like(input)
       grad_reshaped = self.gradInput.reshape(batch_size, channels,
                                            output_h, k, output_w, k)

       # Создаем индексы для распределения градиентов
       b_idx = np.repeat(np.arange(batch_size), channels * output_h * output_w)
       c_idx = np.tile(np.repeat(np.arange(channels), output_h * output_w), batch_size)
       h_idx = np.tile(np.repeat(np.arange(output_h), output_w), batch_size * channels)
       w_idx = np.tile(np.arange(output_w), batch_size * channels * output_h)

       # Преобразуем линейные индексы в координаты в окне пулинга
       pool_h = self.max_indices // k
       pool_w = self.max_indices % k

       # Распределяем градиенты в позиции максимумов
       flat_grad = gradOutput.ravel()
       grad_reshaped[b_idx, c_idx, h_idx, pool_h.ravel(),
                    w_idx, pool_w.ravel()] = flat_grad

       return self.gradInput

   def __repr__(self):
       return f'MaxPool2d, kern {self.kernel_size}, stride {self.kernel_size}'


Разберем ключевые моменты реализации:

1. Прямой проход (`updateOutput`):
```python
# Преобразование входа
x_reshaped = input.reshape(batch_size, channels,
                          output_h, self.kernel_size,
                          output_w, self.kernel_size)
x_reordered = x_reshaped.transpose(0, 1, 2, 4, 3, 5)

 # Максимумы и их индексы
self.output = x_reordered.max(axis=(4, 5))
self.max_indices = x_reordered.reshape(batch_size, channels,
                                     output_h, output_w, -1).argmax(axis=-1)
```

2. Обратный проход (`updateGradInput`):
```python
  # Инициализация градиента
self.gradInput = np.zeros_like(input)
grad_input_reshaped = self.gradInput.reshape(batch_size, channels,
                                           output_h, self.kernel_size,
                                           output_w, self.kernel_size)

 # Распределение градиентов
kernel_h = self.max_indices // self.kernel_size
kernel_w = self.max_indices % self.kernel_size
grad_input_reshaped[batch_idx, channel_idx, height_idx, kernel_h,
                   width_idx, kernel_w] = gradOutput
```

Особенности реализации:

1. Space-to-batch преобразование:
   - Переформатирует входные данные для эффективного пулинга
   - Позволяет использовать векторизованные операции numpy

2. Сохранение индексов:
   - Запоминаем позиции максимальных значений
   - Используем их при обратном проходе
   - Эффективнее, чем повторный поиск максимумов

3. Ограничения:
   - Шаг равен размеру ядра
   - Входные размеры должны делиться на размер ядра
   - Нет поддержки padding

4. Векторизация:
   - Использование numpy для эффективных вычислений
   - Минимум циклов в коде

Типичное использование:
```python
 # Создание слоя
pool = MaxPool2d(kernel_size=2)

 # Прямой проход
x = np.random.randn(32, 64, 28, 28)  # [batch, channels, height, width]
out = pool.updateOutput(x)  # [32, 64, 14, 14]

 # Обратный проход
grad_output = np.random.randn(32, 64, 14, 14)
grad_input = pool.updateGradInput(x, grad_output)  # [32, 64, 28, 28]
```

### Flatten layer
Just reshapes inputs and gradients. It's usually used as proxy layer between Conv2d and Linear.

Сглаживающий слой

Просто изменяет форму входных данных и градиентов. Обычно используется в качестве промежуточного слоя между Conv2d и Linear.


In [22]:
class Flatten(Module):
    def __init__(self):
        """
        Инициализация слоя Flatten.
        Не требует параметров.
        """
        super(Flatten, self).__init__()

    def updateOutput(self, input):
        """
        Прямой проход: преобразует вход в двумерный массив.

        Например:
        [batch_size, channels, height, width] -> [batch_size, channels*height*width]

        Параметры:
        input: многомерный массив с первым измерением batch_size

        Возвращает:
        output: двумерный массив [batch_size, features]
        """
        # Сохраняем размер батча (первое измерение)
        # -1 автоматически вычисляет второе измерение
        self.output = input.reshape(len(input), -1)
        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Обратный проход: восстанавливает исходную форму градиента.

        Параметры:
        input: исходный вход (нужен только для получения формы)
        gradOutput: градиент в плоской форме [batch_size, features]

        Возвращает:
        gradInput: градиент в исходной форме input.shape
        """
        # Восстанавливаем исходную форму для градиента
        self.gradInput = gradOutput.reshape(input.shape)
        return self.gradInput

    def __repr__(self):
        return "Flatten"

Когда использовать:


*   После сверточных слоев перед полносвязными
*  Когда нужно преобразовать многомерные данные в вектор
*  В архитектурах типа CNN + MLP


Важные замечания:

* Сохраняет размер батча (первое измерение)
* Объединяет все остальные измерения в одно
* Не влияет на значения, только на форму данных
* Является полностью обратимым преобразованием